In [3]:
! pip3 install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [36]:
import fitz  # PyMuPDF
import requests
import os, json
pdf_url = "https://people.eecs.berkeley.edu/~vazirani/algorithms/chap2.pdf"
pdf_filename = "chap2.pdf"
response = requests.get(pdf_url)
if response.status_code == 200:
    with open(pdf_filename, 'wb') as pdf_file:
        pdf_file.write(response.content)
else:
    print("Failed to download the PDF file")
    exit()

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(pdf_filename)

In [37]:
pdf_json = {
    "content": pdf_text
}


In [38]:
json_filename = "chap2.json"
with open(json_filename, 'w') as json_file:
    json.dump(pdf_json, json_file, indent=4)


In [39]:
os.remove(pdf_filename)

print(f"PDF content has been converted to JSON and saved to {json_filename}")


PDF content has been converted to JSON and saved to chap2.json


In [33]:
with open('chap2.json', 'r') as file:
    data = json.load(file)

In [41]:
def split_text(text, max_length=21000):
    sentences = text.split('.')
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_length:
            chunk += sentence + '.'
        else:
            chunks.append(chunk)
            chunk = sentence + '.'
    chunks.append(chunk)  # add the last chunk
    return chunks

chunks = split_text(data['content'])
len(chunks)

1

In [31]:
import os
import json

from octoai.client import OctoAI
from octoai.text_gen import ChatMessage


# Get the OctoAI API token from the environment
OCTOAI_API_TOKEN = os.getenv("OCTOAI_API_TOKEN")
if not OCTOAI_API_TOKEN:
    raise ValueError("No OCTOAI_API_TOKEN found in environment variables")

# Initialize OctoAI client
client = OctoAI(api_key=OCTOAI_API_TOKEN)


def get_answer_from_chunk(chunk,question):
    completion = client.text_gen.create_chat_completion(
    model="qwen2-7b-instruct",
    messages=[
        ChatMessage(
            role="system",
            content="You are a professor understand the content of the PDF and explain it to a student.",
        ),
        ChatMessage(role="system", content=question),
        ChatMessage(role="user", content= chunk),
    ],
    max_tokens=512,
    )

    return json.dumps(completion.dict(), indent=2)

question = "What is the content of the PDF?"



In [32]:
i = 0
data = []
for chunk in chunks:
    answer = get_answer_from_chunk(chunk, question)
    data.append(answer)
    print(i)
    i += 1
    print("------------------------------------------------------------------------------------------------")
    print(answer)
    
    



0
------------------------------------------------------------------------------------------------
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The content:\n\nThis text is an introduction to algorithms, a very specific aspect focusing on the efficiency and importance of algorithms in computing as a method to negate the problems that the pace of advances in microelectronics and chip design could not alone overcome. The rapid advancement in semiconductor technology (tcMoore) led to powerful machines but didn't address complexities and challenges.\n\nThe text suggests that the management of data and computation, which revolves around algorithms, is now crucial in driving computerics. It refers to Gutenberg and his invention of movable type as a revolutionary step though, according to some, the algorithmic aspect was at least equally pivotal in bringing scientific and technological revolution and ind

In [10]:
# # data_json = json.dumps(data, indent=2)

# # # Print the JSON data
# # print(data_json)

# # # Parse the JSON string
# # data = json.loads(data)

# # # Extract the "content" key
# # content = data['choices'][0]['message']['content']

# # # Print the extracted content
# # print(content)
# print(data)
# for json_string in data:
#     data1 = json.loads(json_string)
#     # content = data1['choices'][0]['message']['content']
#     print(content)

In [16]:
def extract_contents(json_strings):
    contents = []
    for json_string in json_strings:
        # Parse the JSON string
        data = json.loads(json_string)
        # Extract the content
        content = data['choices'][0]['message']['content']
        contents.append(content)
    return contents

# Extract contents
extracted_contents = extract_contents(data)
final_content = ""
# Print results
for content in extracted_contents:
    #print(content)
    final_content += content

The document you've provided is an introduction and an overview of the power of algorithms in the context of computer science, specifically in relation to the development and impact of the positional decimal system. It covers several key points:

1. The two main technological underpinnings that have driven the computer revolution: hardware advancements (like fast and increasingly powerful chip design), and efficient algorithms.
   
2. The importance of the advent of the decimal positional system and its algorithms, which enabled science, technology, industry, and commerce to flourish. Al Khwarizmi’s work, written in Arabic in the 9th century, was crucial in spreading these ideas from Asia to Europe.

3. The Italian mathematician Fibonacci's role in popularizing the use of the decimal system and his famous Fibonacci sequence, discussed not only for its numerical beauty but also for its computational applications and its "favorite" role in computer science.

4. The inefficiency of a naiv

In [25]:
answer_to_output = get_answer_from_chunk(final_content, question)
print(answer_to_output)


{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The PDF you've described is a foundational document meant to introduce students to the critical role of algorithms in modern technology, centering on the evolution of the positional decimal system and its broader influence across scientific and commercial fields. Let's dive deeper into each key point laid out for a comprehensive understanding:\n\n1. **The Computer Revolution**: The document emphasizes the dual importance of hardware advancements and algorithms in driving innovation. It illustrates that advancements in hardware, particularly in the compactness and speed of chip design, enable remarkable computational capabilities. However, alongside hardware, algorithms are crucial for managing tasks efficiently, directing how data and instructions are processed to achieve optimal results.\n\n2. **The Positional Decimal System**: This system, champione

In [26]:
import re
pattern = r'"content":\s*"([^"]*?)"'
match = re.search(pattern, answer_to_output, re.DOTALL)

if match:
    content = match.group(1)
    print(content)
else:
    print("Content not found")

The PDF you've described is a foundational document meant to introduce students to the critical role of algorithms in modern technology, centering on the evolution of the positional decimal system and its broader influence across scientific and commercial fields. Let's dive deeper into each key point laid out for a comprehensive understanding:\n\n1. **The Computer Revolution**: The document emphasizes the dual importance of hardware advancements and algorithms in driving innovation. It illustrates that advancements in hardware, particularly in the compactness and speed of chip design, enable remarkable computational capabilities. However, alongside hardware, algorithms are crucial for managing tasks efficiently, directing how data and instructions are processed to achieve optimal results.\n\n2. **The Positional Decimal System**: This system, championed by mathematician Al Khwarizmi in the 9th century and further popularized by Fibonacci, is lauded for establishing a common ground for m